<a href="https://colab.research.google.com/github/verma-satyam/ML-Coding-Ninjas/blob/main/8.%20Naive%20Bayes/1_Naive_Bayes_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Naive Bayes

In [39]:
import numpy as np

In [40]:
def fit(X_train,Y_train):
  result = {}
  class_values = set(Y_train)
  for current_class in class_values:
    result[current_class] = {}
    result['total_data'] = len(Y_train)

    X_train_current = X_train[Y_train == current_class]
    Y_train_current = Y_train[Y_train == current_class]

    num_features = X_train.shape[1]
    result[current_class]['total_count'] = len(Y_train_current)

    for j in range(num_features):
      result[current_class][j] = {}
      all_possible_values = set(X_train[:,j])

      for current_value in all_possible_values:
        result[current_class][j][current_value] = (X_train_current[:,j] == current_value).sum()

  return result

In [41]:
def prob(result,x,current_class):
  output = np.log(result[current_class]['total_count'])-np.log(result['total_data'])
  num_features = len(result[current_class].keys()) - 1
  for j in range(num_features):
    xj = x[j]
    output+=np.log(result[current_class][j][xj] +1)-np.log(result[current_class]['total_count'] + len(result[current_class][j].keys()))
  return output

In [42]:
def predictRow(result, x):
  all_classes = result.keys()
  first_run = True
  for current_class in all_classes:
    if current_class == 'total_data':
      continue
    
    p_current_class = prob(result,x,current_class)
    if(first_run or p_current_class > best_p):
      best_p = p_current_class
      best_class = current_class
    first_run = False
  return best_class,best_p

In [43]:
def predict(result,X_test):
  Y_pred = []
  for x in X_test: # for every row in dataset
    pred_class_x,pred_value = predictRow(result,x)
    Y_pred.append(pred_class_x)
  return Y_pred

## Preprocessing Iris into categorical feature

In [44]:
def makeLabel(column):
  secondLimit = column.mean()
  firstLimit = 0.5*secondLimit
  thirdLimit = 1.5*secondLimit
  for i in range(len(column)):
    if(column[i]<firstLimit):
      column[i] = 0
    elif(column[i]<secondLimit):
      column[i] = 1
    elif(column[i]<thirdLimit):
      column[i] = 2
    else:
      column[i] = 3
  return column

In [45]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [46]:
for i in range(X.shape[-1]):
  X[:,i] = makeLabel(X[:,i])

## Model Trainig and Predictions

In [47]:
from sklearn import model_selection
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(X,Y,test_size = 0.25, random_state=0)

In [48]:
result = fit(X_train,Y_train)
Y_pred = predict(result,X_test)

In [49]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.94      1.00      0.97        16
           2       1.00      0.89      0.94         9

    accuracy                           0.97        38
   macro avg       0.98      0.96      0.97        38
weighted avg       0.98      0.97      0.97        38

[[13  0  0]
 [ 0 16  0]
 [ 0  1  8]]
